<a href="https://colab.research.google.com/github/divyansh1195/Deep-Learning/blob/master/Keras%20Tuner%20Optimization/Fashion_Mnist_Keras_Tuner_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Tuner Optimization on Fashion Minst Dataset

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=b64e168b792200338a108a4df8cf83e5dd45459467744506a28f71fa802992f2
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=ecc8a4082091c927bf877d7786ca91a2ddf4d15aad40542373e4f564d34769e4
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


# Importing the Libraries



In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


# import the libraries
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# Data Loading and Preprocessing 

In [3]:
fashion_mnist=keras.datasets.fashion_mnist#load fashion_mnist dataset from keras

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data() # training and test dataset images

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
# since, the fashion_mnist dataset is a black and white dataset, we need to scale down the images' pixel between 0 and 1.
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape#first image

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

# Model Building

In [8]:
def build_model(hp):  #hp is hyperparameters
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  #softmax activation function of a neural network to normalize the output of a network to a probability distribution over predicted output classes
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

## Model Tuning and Fitting

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
#search for best parameter from build model
#initially, we will test run it only for 2 epochs with 5 trials
tuner_search.search(train_images,train_labels,epochs=2,validation_split=0.1)

Epoch 1/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5512 - accuracy: 0.8084 - val_loss: 0.4284 - val_accuracy: 0.8452
Epoch 2/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3989 - accuracy: 0.8529 - val_loss: 0.3988 - val_accuracy: 0.8567


Epoch 1/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5495 - accuracy: 0.8062 - val_loss: 0.3979 - val_accuracy: 0.8532
Epoch 2/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3977 - accuracy: 0.8519 - val_loss: 0.3989 - val_accuracy: 0.8485


Epoch 1/2
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4099 - accuracy: 0.8515 - val_loss: 0.3120 - val_accuracy: 0.8848
Epoch 2/2
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2722 - accuracy: 0.8995 - val_loss: 0.2893 - val_accuracy: 0.8925


Epoch 1/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4038 - accuracy: 0.8540 - val_loss: 0.3097 - val_accuracy: 0.8857
Epoch 2/2
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2709 - accuracy: 0.9001 - val_loss: 0.2789 - val_accuracy: 0.8965


Epoch 1/2
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3834 - accuracy: 0.8604 - val_loss: 0.2773 - val_accuracy: 0.8957
Epoch 2/2
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2397 - accuracy: 0.9108 - val_loss: 0.2580 - val_accuracy: 0.9102


INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]#best models

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        51264     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,026,346
Trainable params: 3,026,346
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=2)

Epoch 3/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1780 - accuracy: 0.9336 - val_loss: 0.2333 - val_accuracy: 0.9122
Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1288 - accuracy: 0.9519 - val_loss: 0.2672 - val_accuracy: 0.9132
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0878 - accuracy: 0.9686 - val_loss: 0.3016 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0613 - accuracy: 0.9771 - val_loss: 0.3358 - val_accuracy: 0.9130
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0443 - accuracy: 0.9840 - val_loss: 0.3609 - val_accuracy: 0.9130
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0327 - accuracy: 0.9881 - val_loss: 0.4409 - val_accuracy: 0.9117
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0261 - accuracy: 0.9909 - val_loss: 0.4906 - val_accuracy: